<a href="https://colab.research.google.com/github/HYUNMIN-HWANG/Colab/blob/main/DACON_vision2_0212_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


######################################################
# File Load
dataset = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/dirty_mnist_2nd_answer.csv')
y_df =  dataset.iloc[:,:]
print(y_df.shape)  # (50000, 27)

sub = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sample_submission.csv')
print(sub.shape)    # (5000, 27)

######################################################

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5,
    fill_mode='nearest'
)
etc_datagen = ImageDataGenerator(rescale=1./255)

def mymodel () :
    model = Sequential()
    model.add(Conv2D(32, (2,2), padding='same', input_shape=(80, 80, 1), activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(1, activation='sigmoid'))

    return model

#1. DATA
x = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x4.npy')
x_pred = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x_pred4.npy')
print("<==complete load==>")

print(x.shape, x_pred.shape) # (50000, 80, 80, 1) (5000, 80, 80, 1)
x[x < 253] = 0

alphabet = ['a','b','c','d','e','f','g','h','i','j','k']
# alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

val_loss_list = []
val_acc_list = []

for alph in alphabet :  # 한 알파벳씩 검증
    print('<<<<<<<<<<<< ', alph, ' predict start >>>>>>>>>>>>')
    y = y_df.loc[:,alph]
    y_pred_list = []
    y_result = 0

    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=42)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=0.8, shuffle=True, random_state=42)
    print(x_train.shape, x_test.shape, x_valid.shape)  # (32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
    print(y_train.shape, y_test.shape, y_valid.shape)  # (32000,) (10000,) (8000,)

    batch = 50
    train_generator = train_datagen.flow(x_train, y_train, batch_size=batch, seed=2021)
    test_generator = etc_datagen.flow(x_test, y_test, batch_size=batch, seed=2021)
    valid_generator = etc_datagen.flow(x_valid, y_valid)
    pred_generator = etc_datagen.flow(x_pred)

    #2. Modeling
    model = mymodel()

    #3. Compile, Train

    path = './MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_{val_loss:.4f}.hdf5'
    cp = ModelCheckpoint(filepath=path, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    es = EarlyStopping(monitor='val_loss', patience=30, mode='min')
    lr = ReduceLROnPlateau(monitor='val_loss', patience=15, factor=0.4, mode='min')

    model.compile(
        loss='binary_crossentropy', 
        optimizer=Adam(lr=0.05, epsilon=None), 
        metrics=['acc']
    )

    hist = model.fit_generator(
        train_generator, epochs=50,
        steps_per_epoch = len(x_train) // batch ,
        validation_data=valid_generator, callbacks=[es, lr, cp]
    )

    #4. Evaluate, Predict
    loss, acc = model.evaluate(test_generator)
    print("loss : ", loss)
    print("acc : ", acc)  

    val_loss_list.append(loss)
    val_acc_list.append(acc)

    # loss :  
    # acc :  

    y_pred = model.predict(pred_generator)
    # print(y_pred.shape) # (5000, 1)
    y_result= np.where(y_pred < 0.5, 0, 1)
    sub.loc[:,alph] = y_result  # 예측값을 sub 파일에 저장
    print(sub.head())

    print("- ", alph, " predict end -\n")

print("mean loss : ", sum(val_loss_list)/len(alphabet))
print("mean acc : ", sum(val_acc_list)/len(alphabet))

sub.to_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sub_0212_1.csv', index=False)
print(sub.head())

# sub
# score 

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


######################################################
# File Load
dataset = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/dirty_mnist_2nd_answer.csv')
y_df =  dataset.iloc[:,:]
print(y_df.shape)  # (50000, 27)

sub = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sample_submission.csv')
print(sub.shape)    # (5000, 27)

######################################################

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5,
    fill_mode='nearest'
)
etc_datagen = ImageDataGenerator(rescale=1./255)

def mymodel () :
    model = Sequential()
    model.add(Conv2D(32, (2,2), padding='same', input_shape=(80, 80, 1), activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(1, activation='sigmoid'))

    return model

#1. DATA
x = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x4.npy')
x_pred = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x_pred4.npy')
print("<==complete load==>")

print(x.shape, x_pred.shape) # (50000, 80, 80, 1) (5000, 80, 80, 1)
x[x < 253] = 0

# alphabet = ['a','b','c','d','e','f','g','h','i','j','k']
alphabet = ['l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

val_loss_list = []
val_acc_list = []

for alph in alphabet :  # 한 알파벳씩 검증
    print('<<<<<<<<<<<< ', alph, ' predict start >>>>>>>>>>>>')
    y = y_df.loc[:,alph]
    y_pred_list = []
    y_result = 0

    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=42)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=0.8, shuffle=True, random_state=42)
    print(x_train.shape, x_test.shape, x_valid.shape)  # (32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
    print(y_train.shape, y_test.shape, y_valid.shape)  # (32000,) (10000,) (8000,)

    batch = 50
    train_generator = train_datagen.flow(x_train, y_train, batch_size=batch, seed=2021)
    test_generator = etc_datagen.flow(x_test, y_test, batch_size=batch, seed=2021)
    valid_generator = etc_datagen.flow(x_valid, y_valid)
    pred_generator = etc_datagen.flow(x_pred)

    #2. Modeling
    model = mymodel()

    #3. Compile, Train

    path = './MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_{val_loss:.4f}.hdf5'
    cp = ModelCheckpoint(filepath=path, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    es = EarlyStopping(monitor='val_loss', patience=30, mode='min')
    lr = ReduceLROnPlateau(monitor='val_loss', patience=15, factor=0.4, mode='min')

    model.compile(
        loss='binary_crossentropy', 
        optimizer=Adam(lr=0.05, epsilon=None), 
        metrics=['acc']
    )

    hist = model.fit_generator(
        train_generator, epochs=50,
        steps_per_epoch = len(x_train) // batch ,
        validation_data=valid_generator, callbacks=[es, lr, cp]
    )

    #4. Evaluate, Predict
    loss, acc = model.evaluate(test_generator)
    print("loss : ", loss)
    print("acc : ", acc)  

    val_loss_list.append(loss)
    val_acc_list.append(acc)

    # loss :  
    # acc :  

    y_pred = model.predict(pred_generator)
    # print(y_pred.shape) # (5000, 1)
    y_result= np.where(y_pred < 0.5, 0, 1)
    sub.loc[:,alph] = y_result  # 예측값을 sub 파일에 저장
    print(sub.head())

    print("- ", alph, " predict end -\n")

print("mean loss : ", sum(val_loss_list)/len(alphabet))
print("mean acc : ", sum(val_acc_list)/len(alphabet))

sub.to_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sub_0212_1_2.csv', index=False)
print(sub.head())

# sub
# score 

(50000, 27)
(5000, 27)
<==complete load==>
(50000, 80, 80, 1) (5000, 80, 80, 1)
<<<<<<<<<<<<  l  predict start >>>>>>>>>>>>
(32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
(32000,) (10000,) (8000,)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
640/640 [==============================] - 25s 32ms/step - loss: 0.7144 - acc: 0.5303 - val_loss: 0.7149 - val_acc: 0.5361

Epoch 00001: val_loss improved from inf to 0.71488, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.7149.hdf5
Epoch 2/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6920 - acc: 0.5296 - val_loss: 40.7898 - val_acc: 0.5077

Epoch 00002: val_loss did not improve from 0.71488
Epoch 3/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6943 - acc: 0.5249 - val_loss: 0.7130 - val_acc: 0.4608

Epoch 00003: val_loss improved from 0.71488 to 0.71303, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.7130.hdf5
Epoch 4/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6938 - acc: 0.5224 - val_loss: 0.6935 - val_acc: 0.5135

Epoch 00004: val_loss improved from 0.71303 to 0.69347, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6935.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
640/640 [==============================] - 21s 31ms/step - loss: 0.7108 - acc: 0.5280 - val_loss: 0.7029 - val_acc: 0.4666

Epoch 00001: val_loss improved from inf to 0.70293, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.7029.hdf5
Epoch 2/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6930 - acc: 0.5304 - val_loss: 0.6938 - val_acc: 0.5328

Epoch 00002: val_loss improved from 0.70293 to 0.69376, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6938.hdf5
Epoch 3/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6928 - acc: 0.5359 - val_loss: 0.6911 - val_acc: 0.5378

Epoch 00003: val_loss improved from 0.69376 to 0.69113, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6911.hdf5
Epoch 4/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6930 - acc: 0.5299 - val_loss: 0.7137 - val_acc: 0.5206

Epoch 00004: val_loss did not improve from 0.69

...

Epoch 1/50
640/640 [==============================] - 21s 31ms/step - loss: 0.7106 - acc: 0.5232 - val_loss: 1.4214 - val_acc: 0.5505

Epoch 00001: val_loss improved from inf to 1.42139, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_1.4214.hdf5
Epoch 2/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6942 - acc: 0.5230 - val_loss: 0.7369 - val_acc: 0.5505

Epoch 00002: val_loss improved from 1.42139 to 0.73691, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.7369.hdf5
Epoch 3/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6934 - acc: 0.5258 - val_loss: 0.7011 - val_acc: 0.5260

Epoch 00003: val_loss improved from 0.73691 to 0.70110, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.7011.hdf5
Epoch 4/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6971 - acc: 0.5198 - val_loss: 0.9577 - val_acc: 0.5500

Epoch 00004: val_loss did not improve from 0.70

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
640/640 [==============================] - 21s 31ms/step - loss: 0.7055 - acc: 0.5221 - val_loss: 0.6912 - val_acc: 0.5350

Epoch 00001: val_loss improved from inf to 0.69116, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6912.hdf5
Epoch 2/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6945 - acc: 0.5207 - val_loss: 0.6911 - val_acc: 0.5350

Epoch 00002: val_loss improved from 0.69116 to 0.69106, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6911.hdf5
Epoch 3/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6947 - acc: 0.5193 - val_loss: 0.6908 - val_acc: 0.5359

Epoch 00003: val_loss improved from 0.69106 to 0.69081, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6908.hdf5
Epoch 4/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6944 - acc: 0.5248 - val_loss: 0.6907 - val_acc: 0.5350

Epoch 00004: val_loss improved from 0.69081 to 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
640/640 [==============================] - 22s 32ms/step - loss: 0.7123 - acc: 0.5310 - val_loss: 0.6912 - val_acc: 0.5425

Epoch 00001: val_loss improved from inf to 0.69119, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6912.hdf5
Epoch 2/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6936 - acc: 0.5354 - val_loss: 0.7347 - val_acc: 0.4601

Epoch 00002: val_loss did not improve from 0.69119
Epoch 3/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6954 - acc: 0.5244 - val_loss: 0.6902 - val_acc: 0.5425

Epoch 00003: val_loss improved from 0.69119 to 0.69021, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6902.hdf5
Epoch 4/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6960 - acc: 0.5259 - val_loss: 18.7215 - val_acc: 0.5425

Epoch 00004: val_loss did not improve from 0.69021
Epoch 5/50
301/640 [=============>................] - ETA: 9s - loss: 0.6928 - acc: